In [ ]:
import requests
import pandas as pd
import datetime as date

polygon_api_key = open("polygon_io_api_key.txt").read()

ticker = 'aapl'.upper()


In [75]:
last_stock_url = f'https://api.polygon.io/v2/aggs/ticker/{ticker}/prev?adjusted=true&apiKey={polygon_api_key}'
last_stock_request = requests.get(last_stock_url)
last_stock_result = last_stock_request.json()

last_stock_df = pd.DataFrame(last_stock_result['results'])
last_close = last_stock_df['c'][0]

call_url = f'https://api.polygon.io/v3/snapshot/options/{ticker}?strike_price.gte={last_close}&contract_type=call&order=asc&limit=1&sort=strike_price&apiKey={polygon_api_key}'
call_request = requests.get(call_url)
call_result = call_request.json()

put_url = f'https://api.polygon.io/v3/snapshot/options/{ticker}?strike_price.lte={last_close}&contract_type=call&order=desc&limit=1&sort=strike_price&apiKey={polygon_api_key}'
put_request = requests.get(put_url)
put_result = put_request.json()

In [ ]:
#Could be used to evaluate implied volatility into the future

atm = result['results'][0]['details']['strike_price']
#atm = 250

atm_url = f'https://api.polygon.io/v3/snapshot/options/{ticker}?strike_price={atm}&contract_type=call&order=asc&limit=250&sort=expiration_date&apiKey={polygon_api_key}'
atm_request = requests.get(atm_url)
atm_result = atm_request.json()['results']

print(atm_result)

[{'day': {'change': 0, 'change_percent': 0, 'close': 3.13, 'high': 9.35, 'last_updated': 1760126400000000000, 'low': 2.85, 'open': 8.43, 'previous_close': 3.13, 'volume': 8134, 'vwap': 3.9604}, 'details': {'contract_type': 'call', 'exercise_style': 'american', 'expiration_date': '2025-10-17', 'shares_per_contract': 100, 'strike_price': 247.5, 'ticker': 'O:AAPL251017C00247500'}, 'greeks': {'delta': 0.4301607646797087, 'gamma': 0.03964321888685126, 'theta': -0.39224182653151035, 'vega': 0.11642848255273658}, 'implied_volatility': 0.3416997905731155, 'open_interest': 2794, 'underlying_asset': {'ticker': 'AAPL'}}, {'day': {'change': 0, 'change_percent': 0, 'close': 4.49, 'high': 10.6, 'last_updated': 1760126400000000000, 'low': 4.38, 'open': 9.75, 'previous_close': 4.49, 'volume': 1606, 'vwap': 5.5013}, 'details': {'contract_type': 'call', 'exercise_style': 'american', 'expiration_date': '2025-10-24', 'shares_per_contract': 100, 'strike_price': 247.5, 'ticker': 'O:AAPL251024C00247500'}, 'g